This code enables the loading of all markerless and marker-based data and their comparison through:
- plotting joint coordinates
- computing classical descriptive statistics

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import numpy as np

In [ ]:
path_markers_data = Path('Data', 'Markers')
path_markerless_data = Path('Data', 'Markerless')

In [ ]:
## Define the participants, tasks, joints, and markerless methods you want to load and analyze
participants = ["participant_01", "participant_02"]
tasks = ["gait", "sit-stand", "mmh", "exotic", "dance"]
joints = ['RHip_FE', 'RHip_AA', 'RKnee_FE', 'RAnkle_FE', 'L5S1_FE', 'RShoulder_AA', 'RShoulder_RIE', 'RElbow_FE', 'RElbow_PS', 'RWrist_FE']
markerless_methods = [d.name for d in path_markerless_data.iterdir() if d.is_dir() and d.name != "Template"]

### Data recovery

In [ ]:
markers_data = dict()
markerless_data = dict()
for participant in participants:
    markers_data[participant] = dict()
    markerless_data[participant] = dict()
    for task in tasks:
        name_csv_markers = Path(path_markers_data) / f"{task}_{participant}.csv"
        markers_data[participant][task] = pd.read_csv(name_csv_markers)
        markerless_data[participant][task] = dict()
        for markerless_method in markerless_methods:
            name_csv_markerless = Path(path_markerless_data) / markerless_method / f"{task}_{participant}.csv"
            if name_csv_markerless.exists():
                markerless_data[participant][task][markerless_method] = pd.read_csv(name_csv_markerless)
            else:
                markerless_data[participant][task][markerless_method] = None

### Plotting joint coordinates

In [ ]:
plt.set_cmap('gist_rainbow')
cmap = plt.get_cmap()
colors = [cmap(i) for i in np.linspace(0, 1, len(markerless_method))]

def plot_graph(participant, task, joint, markers_data, markerless_data, markerless_methods, colors):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_title(f"{participant} - {task} - {joint}")
    
    # Markers
    ax.plot(markers_data[participant][task]['Time'], markers_data[participant][task][joint]*180/np.pi, linewidth=3, color='black')
    legend = ['Marqueurs']
    
    # Markerless
    for index, method in enumerate(markerless_methods):
        if isinstance(markerless_data[participant][task][method], pd.DataFrame):
            ax.plot(markerless_data[participant][task][method]['Time'], markerless_data[participant][task][method][joint]*180/np.pi, color=colors[index])
        else:
            ax.plot(np.nan)
        legend.append(method)
    
    ax.legend(legend)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Joint coordinates (°)')
    plt.tight_layout()

for participant in participants:
    for task in tasks:
        for joint in joints:
            plot_graph(participant, task, joint, markers_data, markerless_data, markerless_methods, colors)

plt.show()

### Descriptive statistics

In [ ]:
def stats_descriptives(series1, series2):
    series1 = np.asarray(series1)
    series2 = np.asarray(series2)
    
    valid_mask = ~np.isnan(series1) & ~np.isnan(series2)
    valid_series1 = series1[valid_mask]
    valid_series2 = series2[valid_mask]
    
    # Bias
    bias = np.mean(valid_series2 - valid_series1)

    # 95% confidence interval
    diff = valid_series2 - valid_series1
    std_dev = np.std(diff)
    n = len(valid_series1)
    conf_interval = 1.96 * std_dev / np.sqrt(n) 

    # R²
    sst = np.sum((valid_series2 - np.mean(valid_series2))**2)

    if sst == 0:
        r_squared = np.nan
    else:
        slope, intercept = np.polyfit(valid_series1, valid_series2, 1)
        predictions = slope * valid_series1 + intercept
        ssr = np.sum((valid_series2 - predictions)**2)
        r_squared = 1 - (ssr / sst)

    # RMSD
    rmsd = np.sqrt(np.mean(diff**2))

    return bias, conf_interval, r_squared, rmsd

def normalize_data(data, N=100):
    new_index = np.linspace(0, len(data) - 1, N)
    interpolated_values = np.interp(new_index, np.arange(len(data)), data)
    data_resampled = pd.Series(interpolated_values, index=range(N))
    return data_resampled

In [ ]:
stats_all = []
for participant in participants:
    for task in tasks:
        for joint in joints:
            # Markers
            data_markers = markers_data[participant][task][joint]
            data_markers_resampled = normalize_data(data_markers)
            for index_method, method in enumerate(markerless_methods):
                if isinstance(markerless_data[participant][task][method], pd.DataFrame) and markerless_data[participant][task][method][joint].notna().any():
                    # Markerless
                    data = markerless_data[participant][task][method][joint]
                    data_resampled = normalize_data(data)
                    # Statistics
                    bias, conf_interval, r_squared, rmsd = stats_descriptives(data_markers_resampled, data_resampled)
                else:
                    bias = conf_interval = r_squared = rmsd = np.nan
                stats_all.append([participant, task, joint, method, bias, conf_interval, r_squared, rmsd])
data_stats_descriptives = pd.DataFrame(stats_all, columns=['participant', 'task', 'joint', 'method', 'bias', 'conf_interval', 'r_squared', 'rmsd'])
data_stats_descriptives.to_csv('Data/stats_descriptives.csv', index=False)